In [12]:
import os
import sys
import re
from tqdm import tqdm
from collections import OrderedDict

_ROOT_PATH = "/home/jiangpeiwen2/jiangpeiwen2/TKGT"
sys.path.insert(0, _ROOT_PATH)

from utils import load_data, save_data,online_local_chat

In [14]:

from KGs.dataset_KGs.cpl import Plaintiff_claim_attributes, cpl_keyword
from date_capture import CplDocumentRatioTextCapture, CplDocumentMoneyTextCapture, CplDocumentDateTextCapture
from test_utils import load_mat, compare, find_context_from_prompt, cpl_post_process_simple, index_after_shuffle, eval_simple
from Hybird_RAG.retriever.retrieve_hybrid import CPL_Hybrid_Retriever

/home/jiangpeiwen2/jiangpeiwen2/miniconda3/envs/llamaindex/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from modelscope.models import Model
from modelscope.pipelines import pipeline
# Version less than 1.1 please use TextRankingPreprocessor
from modelscope.preprocessors import TextRankingTransformersPreprocessor
from modelscope.utils.constant import Tasks

import os
import torch



/home/jiangpeiwen2/jiangpeiwen2/miniconda3/envs/llamaindex/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

model_dict = []
for gpu in range( 8 ):
    #for time_ in range(7):
    temp = []
    for i in range(7):
        # os.environ["CUDA_VISIBLE_DEVICES"]  = str(gpu)
        torch.cuda.set_device(gpu)
        embed_model = HuggingFaceEmbedding(
            model_name="/home/jiangpeiwen2/jiangpeiwen2/LlamaIndex/model/sentence-transformer"
        )
        rerank_pipeline = pipeline(task=Tasks.text_ranking, model='damo/nlp_rom_passage-ranking_chinese-base', model_revision='v1.1.0', device=f"cuda:{gpu}")
        #model_dict.append( embed_model )
        temp.append( (embed_model, rerank_pipeline) )
    model_dict.append( temp )

/home/jiangpeiwen2/jiangpeiwen2/miniconda3/envs/llamaindex/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
2024-11-06 23:04:32,175 - modelscope - INFO - Use user-specified model revision: v1.1.0
2024-11-06 23:04:32,422 - modelscope - INFO - initiate model from /home/jiangpeiwen2/.cache/modelscope/hub/damo/nlp_rom_passage-ranking_chinese-base
2024-11-06 23:04:32,423 - modelscope - INFO - initiate model from location /home/jiangpeiwen2/.cache/modelscope/hub/damo/nlp_rom_passage-ranking_chinese-base.
2024-11-06 23:04:32,425 - modelscope - INFO - initialize model from /home/jiangpeiwen2/.cache/modelscope/hub/damo/nlp_rom_passage-ranking_chinese-base
2024-11-06 23:0

KeyboardInterrupt: 

In [7]:
auto_inputs = {
            'source_sentence': ["你好"],
            'sentences_to_compare': ["你好"]
        }
model_dict[0][1](input=auto_inputs)

/home/jiangpeiwen2/jiangpeiwen2/miniconda3/envs/llamaindex/lib/python3.10/site-packages/transformers/modeling_utils.py:1126: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


{'scores': [0.4277496635913849]}

In [18]:
_MODE = "debug"

if _MODE == "test":
    predict_lists, prompt_lists, label_lists, texts, core_ruled_text, entity_lists, context_lists = load_mat( _MODE )
    
elif _MODE == "train":
    core_ruled_text, texts, label_lists, entity_lists = load_mat( _MODE )
elif _MODE == "debug":
    pair = load_data( "/home/jiangpeiwen2/jiangpeiwen2/TKGT/Hybird_RAG/evaluation/eval_results/history/cpl_data_cell_all_eval_pair_list.pickle", "pickle")
    label_lists, predict_lists = pair[0], pair[1]   # 1456个测试样本，每个样本的prompt为0或12或24
    prompt_lists = load_data( "/home/jiangpeiwen2/jiangpeiwen2/TKGT/Hybird_RAG/temp/cpl/cpl_data_try_cell_all_prompt_list.pickle", "pickle")
    texts = load_data( "/home/jiangpeiwen2/jiangpeiwen2/TKGT/data/cpl/test.text" , "text")
    core_ruled_text = CPL_Hybrid_Retriever.cpl_batch_process( "test")
    # 比较预测结果
    
    def compare_predict( predict_lists, label_lists ):
        prompt_count = 0
        label_count = 0

        same_count = []
        label_not = []
        same_count_pre = []
        for i in range( len(predict_lists)):
            prompt_count += len( predict_lists[i] )
            label_count += len( label_lists[i] )
            for item in label_lists[i]:
                # 两者一样
                if item in predict_lists[i]:
                    same_count.append( item )
                else:   
                    label_not.append( item )
            for item in predict_lists[i]:
                # 如果预测中的不在一样的里面
                if item not in same_count:
                    same_count_pre.append( item )
        print(f"label有{label_count}个，提示词有{prompt_count}个，一样的有{len(same_count)}个")
        return same_count, label_not, same_count_pre
    same_count, label_not, same_count_pre = compare_predict( predict_lists, label_lists )
    """"""
elif _MODE == "post_process":
    predict_lists = load_data( "/home/jiangpeiwen2/jiangpeiwen2/TKGT/Hybird_RAG/temp/cpl/cpl_data_try_cell_all_predict_list.pickle", "pickle")
    column_lists = load_data( "/home/jiangpeiwen2/jiangpeiwen2/TKGT/Hybird_RAG/temp/cpl/cpl_data_try_cell_all_column_list.pickle", "pickle")
    label_lists = load_data( "/home/jiangpeiwen2/jiangpeiwen2/TKGT/Hybird_RAG/temp/cpl/cpl_data_try_cell_all_label_list.pickle", "pickle")
    new_predicts_list = []
    new_labels_list = []
    for i in range(len(predict_lists)):
        temp = set()
        for j in range(len( predict_lists[i])):
            temp.add( (column_lists[i][j][0], column_lists[i][j][1], predict_lists[i][j]))
        ret_label_set, ret_predict_set = cpl_post_process_simple( temp, label_lists[i])
        new_predicts_list.append( ret_predict_set )
        new_labels_list.append( ret_label_set )
    save_data( (new_labels_list, new_predicts_list),"/home/jiangpeiwen2/jiangpeiwen2/TKGT/Hybird_RAG/evaluation/eval_results/cpl_data_cell_all_eval_pair_list.pickle")

label有2364个，提示词有1928个，一样的有1231个


In [22]:
prompt_lists[0]

["\n你是一名信息抽取专家，请你从法院判决文书上下文中抽取问题中要求的日期、金额或利率。请注意区分法院、原告、被告三者对同一事实的不同主张，不要张冠李戴。\n（1）首先，你需要判断上下文中的信息是否包含问题要求的特定对象对某个案件事实的陈述，如果没有（很有可能是无效的上下文），请大胆回答<NOT FOUND>。\n（2）其次，你要判断该数值是否是问题中 主体角色 的主张。例如，上下文中若只有原告主张归还500元，而问题是被告主张归还多少元时，不应该回答500。\n\n相关上下文：['借款金额50万元整', '本院认为，原告贾瑞起与被告李安新之间最初签订《合作经营协议书》，原告所付款项50万元虽名为投资款，但根据该协议书约定，原告有权收回所付款项，并确保获得相对明确的收益，且不参与公司的任何管理，不承担任何亏损，双方的行为和资金往来更符合借贷的特征', '同日，被告李安新出具了借据，载明：今借到贾瑞起50万元整（本金），借款期限自2015年12月3日起至2016年5月31日止，注：此款于2013年5月31日收到', '乙方两年内发出的所有快件（不限量）按照总公司内部价格结算，甲方给乙方韵达结算账号和密码，甲方不加任何费用，乙方自购面单价格每张2.10元，乙方所发出每票快件甲方收取0.70元']\n问题：法院 (上海市闵行区人民法院) 判定 需返回本金总额（元）？\n注意：需返回本金总额指的是被告一共需要归还给原告的金额，单位为元，例如若为50万元，需要转换为500000元的形式。\n答案：\n",
 "\n你是一名信息抽取专家，请你从法院判决文书上下文中抽取问题中要求的日期、金额或利率。请注意区分法院、原告、被告三者对同一事实的不同主张，不要张冠李戴。\n（1）首先，你需要判断上下文中的信息是否包含问题要求的特定对象对某个案件事实的陈述，如果没有（很有可能是无效的上下文），请大胆回答<NOT FOUND>。\n（2）其次，你要判断该数值是否是问题中 主体角色 的主张。例如，上下文中若只有原告主张归还500元，而问题是被告主张归还多少元时，不应该回答500。\n\n相关上下文：['原告贾瑞起向本院提出诉讼请求：判令两被告归还原告借款本金256,000元', '事实和理由：被告因公司资金周转紧张，于2013年5月31日向原告借款50万元，当时双方未签订书面借款协议，也没

In [ ]:
predict_lists[1]

{('上海市闵行区人民法院', '借款实际交付时间1', '2013-05-31'),
 ('上海市闵行区人民法院', '已还款金额1（元）', '100000'),
 ('贾瑞起', '借款实际交付时间1', '2013-05-31'),
 ('贾瑞起', '借款实际交付金额1（元）', '500000'),
 ('贾瑞起', '初始约定的借款金额（元）', '500000'),
 ('贾瑞起', '需返回本金总额（元）', '256000')}

In [19]:
label_lists[0]

{('上海市闵行区人民法院', '借款实际交付时间1', '2013-05-31'),
 ('上海市闵行区人民法院', '借款实际交付金额1（元）', '500000'),
 ('上海市闵行区人民法院', '初始约定的借款金额（元）', '500000'),
 ('上海市闵行区人民法院', '初始约定的还款日期', '2015-12-03'),
 ('上海市闵行区人民法院', '已还款金额1（元）', '244000'),
 ('上海市闵行区人民法院', '需返回本金总额（元）', '256000'),
 ('贾瑞起', '借款实际交付时间1', '2013-05-31'),
 ('贾瑞起', '借款实际交付金额1（元）', '500000'),
 ('贾瑞起', '初始约定的借款金额（元）', '500000'),
 ('贾瑞起', '已还款金额1（元）', '244000'),
 ('贾瑞起', '需返回本金总额（元）', '256000')}

In [6]:
prompt_lists[0]

["\n你是一名信息抽取专家，请你从法院判决文书上下文中抽取问题中要求的日期、金额或利率。请注意区分法院、原告、被告三者对同一事实的不同主张，不要张冠李戴。\n（1）首先，你需要判断上下文中的信息是否包含问题要求的特定对象对某个案件事实的陈述，如果没有（很有可能是无效的上下文），请大胆回答<NOT FOUND>。\n（2）其次，你要判断该数值是否是问题中 主体角色 的主张。例如，上下文中若只有原告主张归还500元，而问题是被告主张归还多少元时，不应该回答500。\n\n相关上下文：['借款金额50万元整', '本院认为，原告贾瑞起与被告李安新之间最初签订《合作经营协议书》，原告所付款项50万元虽名为投资款，但根据该协议书约定，原告有权收回所付款项，并确保获得相对明确的收益，且不参与公司的任何管理，不承担任何亏损，双方的行为和资金往来更符合借贷的特征', '同日，被告李安新出具了借据，载明：今借到贾瑞起50万元整（本金），借款期限自2015年12月3日起至2016年5月31日止，注：此款于2013年5月31日收到', '乙方两年内发出的所有快件（不限量）按照总公司内部价格结算，甲方给乙方韵达结算账号和密码，甲方不加任何费用，乙方自购面单价格每张2.10元，乙方所发出每票快件甲方收取0.70元']\n问题：法院 (上海市闵行区人民法院) 判定 需返回本金总额（元）？\n注意：需返回本金总额指的是被告一共需要归还给原告的金额，单位为元，例如若为50万元，需要转换为500000元的形式。\n答案：\n",
 "\n你是一名信息抽取专家，请你从法院判决文书上下文中抽取问题中要求的日期、金额或利率。请注意区分法院、原告、被告三者对同一事实的不同主张，不要张冠李戴。\n（1）首先，你需要判断上下文中的信息是否包含问题要求的特定对象对某个案件事实的陈述，如果没有（很有可能是无效的上下文），请大胆回答<NOT FOUND>。\n（2）其次，你要判断该数值是否是问题中 主体角色 的主张。例如，上下文中若只有原告主张归还500元，而问题是被告主张归还多少元时，不应该回答500。\n\n相关上下文：['原告贾瑞起向本院提出诉讼请求：判令两被告归还原告借款本金256,000元', '事实和理由：被告因公司资金周转紧张，于2013年5月31日向原告借款50万元，当时双方未签订书面借款协议，也没

In [ ]:
# 获取提示词和label：return ret_list, new_label, first_column, contexts
prompt_temp_instruction = """
你是一名信息抽取专家，请你从法院判决文书上下文中抽取问题中要求的日期、金额或利率。请注意区分法院、原告、被告三者对同一事实的不同主张，不要张冠李戴。
（1）首先，你需要判断上下文中的信息是否包含问题要求的特定对象对某个案件事实的陈述，如果没有（很有可能是无效的上下文），请大胆回答<NOT FOUND>。
（2）其次，你要判断该数值是否是问题中 主体角色 的主张。例如，上下文中若只有原告主张归还500元，而问题是被告主张归还多少元时，不应该回答500。
"""
prompt_temp_input = """
相关上下文：{CONTEXT}
问题：{ROLE} {PREDICATE} {ATTRIBUTE}？
注意：{ATTENTION}
答案：
"""

PREDICATEs = {
    "法院": "判定",
    "原告": "诉称",
    "被告": "辩称",
}

def get_ruled_context_( predicate, entity, attr, hybrid_retriever):
    # 获取当前角色类型和名称
    entity_name = None
    entity_type = None
    for word in ["法院", "原告", "被告"]:
        if word in entity:
            entity_type = word
            entity_name = entity.split(" ")[1].replace("(", "").replace(")", "").strip()
            break
    # 获取ruled_text中对应的部分
    query = entity_type + predicate + attr + "？"
    context = hybrid_retriever.hybrid_retrieve( query, entity, attr)
    if entity_type == "被告":
        filter_context = []
        for line in context:
            if entity_name in line:
                filter_context.append( line )
        context = filter_context
    return context, cpl_keyword[attr]["warning"]

# 从label集合中获取答案，服务于ft
def get_ans_from_label( label, entity, attr ):
    entity_name = None
    entity_type = None
    for word in ["法院", "原告", "被告"]:
        if word in entity:
            entity_type = word
            entity_name = entity.split(" ")[1].replace("(", "").replace(")", "").strip()
            break
    if entity_name != None:
        for item in label:
            if item[0]==entity_name and item[1]==attr:
                return item[2]
    return "<NOT FOUND>"

def get_ready_context( contexts_ready, attr, entity_name ):
    attr_clean = attr if "1" not in attr else attr.replace("1", "")
    flag = 0
    for line in contexts_ready:
        clear_line = line.split("\n\n问题：")[1].split("\n主体角色")[0]
        if attr_clean in line and entity_name in line:
            flag = 1
            context = line
            break
    if flag == 0:
        context = []
    return context, cpl_keyword[attr]["warning"]


# 对单个文档，更新prompt和对应的label，返回可以直接评估的集合格式
def get_prompts_labels( entity_list, index, Plaintiff_claim_attributes, label, core_ruled_text, _entity_type=None, contexts_ready = None):
    ret_list = []
    first_column = []
    contexts = []

    # 处理标签
    new_label = set()
    for item in label:
        for ent in entity_list:
            try:
                if item[0] in ent:
                    item_type = ent
                    break
            except:
                raise Exception( f"{item}, {ent}")
        if _entity_type == None or _entity_type in item_type:
            new_label.add( item )

    attr_list = list(Plaintiff_claim_attributes.keys())
    # 遍历所有属性
    for i in tqdm(range(len(attr_list)), desc=f"正在处理第{index}个文档的属性"):
        attr = attr_list[i]
        if attr != "姓名名称":
            if "（元）" in attr:
                attr_typ = "money"
            elif "日期" in attr or "时间" in attr:
                attr_typ = "date"
            elif "（百分比或元）" in attr:
                attr_typ = "ratio"
            

            for j, entity in enumerate(entity_list):
                # 遍历形式['法院 (上海市闵行区人民法院)', '出借人（原告） (陈达奋)', '借款人（被告） (陈银森)', '借款人（被告） (郑丽珍)']
                if _entity_type==None or _entity_type in entity:
                    # entity_type为两个字
                    for word in ["法院", "原告", "被告"]:
                        if word in entity:
                            entity_type = word
                            entity_name = entity.split(" ")[1].replace("(", "").replace(")", "").strip()
                            break
                    if contexts_ready == None:
                        hybrid_retriever = CPL_Hybrid_Retriever( core_ruled_text[index], attr_typ, entity_type )
                    if contexts_ready != None:
                        context, attention = get_ready_context( contexts_ready, attr, entity_name )
                    else:
                        context, attention = get_ruled_context_( PREDICATEs[entity_type], entity, attr, hybrid_retriever)
                    # context, attention = get_ruled_context(  entity, attr, label)
                    if len(context)!=0:
                        contexts.append( context )
                        first_column.append( [entity_name, attr])
                        if "1" in attr:
                            _attr = attr.replace("1", "")
                        else:
                            _attr = attr
                        if _MODE == "test":
                            prompt = prompt_temp_instruction + prompt_temp_input.format( ROLE = entity, PREDICATE=PREDICATEs[entity_type], 
                                                                                        ATTRIBUTE=_attr, CONTEXT=context, ATTENTION=attention)
                        else:
                            prompt = {
                                "instruction" : prompt_temp_instruction,
                                "input" : prompt_temp_input.format( ROLE = entity, PREDICATE=PREDICATEs[entity_type], ATTRIBUTE=_attr, CONTEXT=context, ATTENTION=attention ),
                                "output": get_ans_from_label( label, entity, attr )     # 从label获取答案
                            }
                        ret_list.append( prompt )
    if contexts_ready!=None:
        contexts = contexts_ready
    return ret_list, new_label, first_column, contexts

In [13]:
new_prompt_lists = []
new_label_lists = []
first_column_lists = []
contexts_lists = []
# for i in range(len(label_lists)):
i = 0
ret_list, new_label, first_column, contexts = get_prompts_labels( entity_lists[i], i, Plaintiff_claim_attributes, 
                                                                label_lists[i], core_ruled_text, _entity_type=None )
                                                                #, contexts_ready=context_lists[i])
new_prompt_lists.append( ret_list )
new_label_lists.append( new_label )
first_column_lists.append( first_column )
contexts_lists.append( contexts )

def compare_prompt( first_column_lists, new_label_lists ):
    prompt_count = 0
    label_count = 0
    same_count = 0
    role_dict = {}
    not_label = []
    for i in range( len(first_column_lists)):
        prompt_count += len( first_column_lists[i] )
        label_count += len( new_label_lists[i] )
        # 遍历标签
        for item in new_label_lists[i]:
            role = item[0]
            attr = item[1]
            if role not in role_dict:
                role_dict[role] = set()
            role_dict[role].add( attr )
        # 遍历提示词
        for item in first_column_lists[i]:
            role = item[0]
            attr = item[1]
            if role in role_dict and attr in role_dict[role]:
                same_count +=1                                  # 提示词中和标签一致的结构
            else:
                not_label.append( item )
    print(f"label有{label_count}个，提示词有{prompt_count}个，一样的有{same_count}个")
    return not_label


# not_label  = compare_prompt( first_column_lists, new_label_lists )      # label有2309个，提示词有3153个，一样的有2084个

正在处理第0个文档的属性:   0%|          | 0/19 [00:00<?, ?it/s]The tokenizer parameter is deprecated and will be removed in a future release. Use a stemmer from PyStemmer instead.
2024-11-06 20:24:11,329 - modelscope - INFO - Use user-specified model revision: v1.1.0
2024-11-06 20:24:11,587 - modelscope - INFO - initiate model from /home/jiangpeiwen2/.cache/modelscope/hub/damo/nlp_rom_passage-ranking_chinese-base
2024-11-06 20:24:11,588 - modelscope - INFO - initiate model from location /home/jiangpeiwen2/.cache/modelscope/hub/damo/nlp_rom_passage-ranking_chinese-base.
2024-11-06 20:24:11,590 - modelscope - INFO - initialize model from /home/jiangpeiwen2/.cache/modelscope/hub/damo/nlp_rom_passage-ranking_chinese-base
2024-11-06 20:24:12,768 - modelscope - INFO - head has no _keys_to_ignore_on_load_missing
/home/jiangpeiwen2/jiangpeiwen2/miniconda3/envs/llamaindex/lib/python3.10/site-packages/modelscope/utils/checkpoint.py:550: FutureWarning: You are using `torch.load` with `weights_only=False` (t

In [ ]:
new_prompt_lists[0]

["\n你是一名信息抽取专家，请你从法院判决文书上下文中抽取问题中要求的日期、金额或利率。请注意区分法院、原告、被告三者对同一事实的不同主张，不要张冠李戴。\n（1）首先，你需要判断上下文中的信息是否包含问题要求的特定对象对某个案件事实的陈述，如果没有（很有可能是无效的上下文），请大胆回答<NOT FOUND>。\n（2）其次，你要判断该数值是否是问题中 主体角色 的主张。例如，上下文中若只有原告主张归还500元，而问题是被告主张归还多少元时，不应该回答500。\n\n相关上下文：['借款金额50万元整', '本院认为，原告贾瑞起与被告李安新之间最初签订《合作经营协议书》，原告所付款项50万元虽名为投资款，但根据该协议书约定，原告有权收回所付款项，并确保获得相对明确的收益，且不参与公司的任何管理，不承担任何亏损，双方的行为和资金往来更符合借贷的特征', '同日，被告李安新出具了借据，载明：今借到贾瑞起50万元整（本金），借款期限自2015年12月3日起至2016年5月31日止，注：此款于2013年5月31日收到', '乙方两年内发出的所有快件（不限量）按照总公司内部价格结算，甲方给乙方韵达结算账号和密码，甲方不加任何费用，乙方自购面单价格每张2.10元，乙方所发出每票快件甲方收取0.70元']\n问题：法院 (上海市闵行区人民法院) 判定 需返回本金总额（元）？\n注意：需返回本金总额指的是被告一共需要归还给原告的金额，单位为元，例如若为50万元，需要转换为500000元的形式。\n答案：\n",
 "\n你是一名信息抽取专家，请你从法院判决文书上下文中抽取问题中要求的日期、金额或利率。请注意区分法院、原告、被告三者对同一事实的不同主张，不要张冠李戴。\n（1）首先，你需要判断上下文中的信息是否包含问题要求的特定对象对某个案件事实的陈述，如果没有（很有可能是无效的上下文），请大胆回答<NOT FOUND>。\n（2）其次，你要判断该数值是否是问题中 主体角色 的主张。例如，上下文中若只有原告主张归还500元，而问题是被告主张归还多少元时，不应该回答500。\n\n相关上下文：['原告贾瑞起向本院提出诉讼请求：判令两被告归还原告借款本金256,000元', '事实和理由：被告因公司资金周转紧张，于2013年5月31日向原告借款50万元，当时双方未签订书面借款协议，也没

: 

In [ ]:
save_data( new_prompt_lists, "/home/jiangpeiwen2/jiangpeiwen2/TKGT/Hybird_RAG/temp/cpl/cpl_data_try_cell_all_prompt_list.pickle")
save_data( new_label_lists, "/home/jiangpeiwen2/jiangpeiwen2/TKGT/Hybird_RAG/temp/cpl/cpl_data_try_cell_all_label_list.pickle")
save_data( first_column_lists, "/home/jiangpeiwen2/jiangpeiwen2/TKGT/Hybird_RAG/temp/cpl/cpl_data_try_cell_all_column_list.pickle")
save_data( contexts_lists, "/home/jiangpeiwen2/jiangpeiwen2/TKGT/Hybird_RAG/temp/cpl/cpl_data_try_cell_all_context_list.pickle")

# 分隔符

In [ ]:
# 将label list分成不同字段的list并统计
def split_label( label_lists ):
    split_dict = {
        "list": {},
        "count":{}
    }
    for key in Plaintiff_claim_attributes.keys():
        split_dict["list"][key] = OrderedDict()
        split_dict["count"][key] = 0
    for i in range( len(label_lists) ):
        # 构建本档案的所有三元组情况
        temp_dict = {}
        for item in label_lists[i]:
            if item[1] not in temp_dict:
                temp_dict[item[1]] = set()
            temp_dict[item[1]].add( item )
        # 将本档案加入全局split_dict
        for key in split_dict["list"].keys():
            if key in temp_dict:
                split_dict["list"][key][i] = temp_dict[key]
                split_dict["count"][key] += 1
            else:
                split_dict["list"][key][i] = []
    return split_dict
split_dict_label = split_label( label_lists )
split_dict_predict = split_label( predict_lists )
# 显示哪个文件
# texts[4].split("docx###")[0]
start_interest_label = split_dict_label["list"]["需返回利息计算起始日期"]               # 199-135   label-predict
end_interest_label = split_dict_label["list"]["需返回利息计算截止日期"]                 # 33-26
start_over_interest_label = split_dict_label["list"]["需返回逾期利息计算起始日期"]       # 48-59
end_over_interest_label = split_dict_label["list"]["需返回逾期利息计算截止日期"]         # 5-2
start_default_interest_label = split_dict_label["list"]["需返回违约金计算起始日期"]      # 5-9
end_default_interest_label = split_dict_label["list"]["需返回违约金计算截止日期"]        # 0-1

agreed_pay_label = split_dict_label["list"]["初始约定的还款日期"]                       # 92-101
actual_give_label = split_dict_label["list"]["借款实际交付时间1"]                       # 102-163
actual_pay_label = split_dict_label["list"]["已还款时间1"]                              # 19-11

def get_pairs( part, split_dict_label, split_dict_predict):
    label_part, predict_part = [], []
    for i in range(len( split_dict_label["list"][part]) ):
        label_part.append( split_dict_label["list"][part][i] )
        predict_part.append( split_dict_predict["list"][part][i] )
    return label_part, predict_part

In [ ]:
index = 1
predict_list = predict_lists[index]
#prompt_list = prompt_lists[index]
_label = label_lists[index]

# entity_list = ['法院 (上海市闵行区人民法院)', '出借人（原告） (贾瑞起)', '借款人（被告） (李安新)', '借款人（被告） (汪少敏)']
# prompt_list, new_label, first_column, contexts = get_prompts_labels( entity_list, Plaintiff_claim_attributes, _label)
# 推理
# predicts = online_local_chat( prompt_list )
# 将原始的label和Predict集合准备成字典以备显示使用
roles = {}
for i, sets in enumerate( [_label, predict_list] ):
    for item in sets:
        if item[0] not in roles:
            roles[ item[0] ] = [ set(), set()]  # 标签、预测
        roles[ item[0] ][i].add( item )
roles.keys()

In [ ]:
compare( roles[role][0],  roles[role][1])